# 0. Import Libraries

In [4]:
# !conda install -c rapidsai -c conda-forge -c nvidia \
#     cudf=24.02 python=3.10 cuda-version=11.8

In [14]:
# %load_ext cudf.pandas

Retrieving notices: ...working... done
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - cudf=24.02

Current channels:

  - https://conda.anaconda.org/rapidsai/win-64
  - https://conda.anaconda.org/rapidsai/noarch
  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/nvidia/win-64
  - https://conda.anaconda.org/nvidia/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [7]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize

# Importing libraries
import os
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight') 
# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from pylab import rcParams
from plotly import tools
# import plotly.plotly as py
# from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.figure_factory as ff
import statsmodels.api as sm
from numpy.random import normal, seed
from scipy.stats import norm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error

# 1. Read Dataset

In [8]:
df = pd.read_csv('candidate.csv')

# 2. Primary Data Analysis

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 80 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   time    100000 non-null  object 
 1   f0      100000 non-null  float64
 2   f1      100000 non-null  float64
 3   f2      100000 non-null  float64
 4   f3      100000 non-null  float64
 5   f4      96555 non-null   float64
 6   f5      64334 non-null   float64
 7   f6      64328 non-null   float64
 8   f7      64346 non-null   float64
 9   f8      98900 non-null   float64
 10  f9      98668 non-null   float64
 11  f10     98641 non-null   float64
 12  f11     96484 non-null   float64
 13  f12     96267 non-null   float64
 14  f13     79094 non-null   float64
 15  f14     99747 non-null   float64
 16  f15     99747 non-null   float64
 17  f16     100000 non-null  float64
 18  f17     100000 non-null  float64
 19  f18     100000 non-null  float64
 20  f19     100000 non-null  float64
 21  f20     100

In [10]:
res = df.describe().T
res['CoV'] = res['std']/res['mean']
res.T

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78
count,100000.000000,100000.000000,100000.000000,100000.000000,96555.000000,64334.000000,64328.000000,64346.000000,98900.000000,98668.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,52024.000000,99734.000000,99734.000000
mean,-0.000003,-0.000020,-0.000004,0.000004,88.815905,14.831080,14.515719,14.651709,15.086703,14.071026,...,1.000306,1.219275,1.218314,1.000805,1.219843,1.217659,1.001830,22688.337769,6319.395061,167.161169
std,0.001820,0.003225,0.001788,0.000648,12.894039,4.128563,4.272675,4.209654,3.327054,3.829419,...,0.000266,0.119277,0.119389,0.000629,0.119202,0.119456,0.001337,22031.816466,7177.081966,172.307057
min,-0.026390,-0.088035,-0.029476,-0.013172,60.200000,9.100000,8.710000,8.960000,9.740000,7.260000,...,1.000000,0.998000,0.997300,1.000000,0.998500,0.997300,1.000000,226.625000,0.000000,0.000000
25%,-0.000895,-0.001451,-0.000738,-0.000277,80.530000,11.980000,11.550000,11.720000,13.070000,11.690000,...,1.000100,1.129500,1.128100,1.000400,1.130400,1.127375,1.001000,8011.475000,1935.250000,63.000000
50%,-0.000001,-0.000028,0.000000,0.000004,86.960000,13.920000,13.510000,13.700000,14.375000,13.230000,...,1.000200,1.180700,1.179600,1.000600,1.181500,1.178900,1.001500,16918.762500,3926.000000,120.000000
75%,0.000868,0.001357,0.000705,0.000294,94.880000,16.420000,16.140000,16.220000,16.260000,15.460000,...,1.000400,1.333800,1.333000,1.001000,1.334300,1.332400,1.002300,30312.700000,7818.000000,212.000000
max,0.075657,0.073741,0.022982,0.020598,989.190000,50.930000,51.620000,50.780000,49.110000,60.170000,...,1.009300,1.481100,1.480400,1.025000,1.481100,1.479900,1.034200,377127.750000,99960.000000,5141.000000
CoV,-563.716380,-161.924878,-427.831028,157.061340,0.145177,0.278372,0.294348,0.287315,0.220529,0.272149,...,0.000266,0.097826,0.097996,0.000628,0.097719,0.098103,0.001334,0.971063,1.135723,1.030784


# 3. Get Returns Column

In [11]:
df['return'] = (df['f24'].shift(-78) - df['f24'])/df['f24']

# 4. Data Visualization

In [12]:
import dtale
data = df

# Assigning a reference to a running D-Tale process.
d = dtale.show(df)

In [13]:
d

In [ ]:
# 5. 

In [ ]:
class Model:

    def prepare_features(self, df):
        """
        :param df: this is the data you want to use to prepare the features for your model
        :return: X, a matrix of features (can be a numpy array or a pandas dataframe, your choice!)
        """
        # todo: implement this function - you can use some of the features given to you or you can build a batch of
        #  your own based on the data that you are given.
        # *** PLEASE ENSURE THAT DO NOT INTRODUCE A LOOKAHEAD IN THIS MATRIX ***
        # *** Bonus points for coding a function that tests against lookahead in X ***
        pass

    def fit(self, path_to_train_csv, *args, **kwargs):
        # todo: read train csv
        # todo: do any operation you would like on it

        # todo: prepare features for the model fit
        X = self.prepare_features(some_dataframe)

        # todo: fit your model here - use X (features matrix), y (the target variable) and any other information you
        #  want to use

        # this follows the scikit-learn pattern by returning self
        return self

    def predict(self, path_to_test_csv, *args, **kwargs):
        # todo: read test csv
        # todo: do any operation you would like on it

        # todo: prepare features for the model predict
        X = self.prepare_features(some_dataframe)

        # todo: calculate your model prediction (call it ypred) using X and any other information you want to use

        # this follows the scikit-learn pattern by returning ypred
        return ypred


if __name__ == '__main__':
    train_csv_path = 'put your local path here'
    test_csv_path = 'put your local path here'

    fit_args = []  # todo: populate this as you see fit
    fit_kwargs = {}  # todo: populate this as you see fit
    clf = Model()
    clf.fit(train_csv_path, *fit_args, **fit_kwargs)

    predict_args = []  # todo: populate this as you see fit
    predict_kwargs = {}  # todo: populate this as you see fit
    ypred = clf.predict(test_csv_path, *predict_args, **predict_kwargs)
